In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance

In [2]:
# 데이터 불러오기
df_train = pd.read_csv("src/data/train.csv") 
df_test = pd.read_csv("src/data/test.csv")

# 결측치 제거
df_train = df_train.dropna(axis=1)
df_test = df_test.dropna(axis=1)

# x와 y분류
x = df_train.drop(['ID', 'Y_LABEL'], axis=1)
y = df_train['Y_LABEL']

# test데이터에서 ID값 제거
df_test = df_test.drop(['ID'], axis = 1)

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, stratify=y)

scaler = StandardScaler()
le = LabelEncoder()

# 변수 정규화
def get_values(value):
  return value.values.reshape(-1, 1)

categorical_features = ['COMPONENT_ARBITRARY','YEAR']
test_stage_features = ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR' , 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

for col in x:
  if col not in categorical_features:
    x_train[col] = scaler.fit_transform(get_values(x_train[col]))
    x_valid[col] = scaler.transform(get_values(x_valid[col]))
    if col in df_test.columns:
        df_test[col] = scaler.transform(get_values(df_test[col]))

for col in categorical_features:
  x_train[col] = le.fit_transform(x_train[col])
  x_valid[col] = le.transform(x_valid[col])
  if col in df_test.columns:
      df_test[col] = le.transform(df_test[col])

C:\Users\User\AppData\Local\Temp\ipykernel_6520\1624444302.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train[col] = scaler.fit_transform(get_values(x_train[col]))
C:\Users\User\AppData\Local\Temp\ipykernel_6520\1624444302.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_valid[col] = scaler.transform(get_values(x_valid[col]))
C:\Users\User\AppData\Local\Temp\ipykernel_6520\1624444302.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [3]:
# Xgboost 이진분류 모델 gridCV로 파라미터 튜닝
params = {'n_estimators': [100,200,400,600,800,1000],
        'max_depth' : [3,4,6,8,10,12],
        'eta' : [0.1], 
        'objective' : ['binary:logistic']}

model = XGBClassifier()
model_grid = GridSearchCV(model, param_grid=params, cv=3)
model_grid.fit(x_train, y_train, early_stopping_rounds=30, eval_metric='auc', eval_set = [(x_valid, y_valid)])

[0]	validation_0-auc:0.81345
[1]	validation_0-auc:0.81371
[2]	validation_0-auc:0.81374
[3]	validation_0-auc:0.82792
[4]	validation_0-auc:0.83904
[5]	validation_0-auc:0.83843
[6]	validation_0-auc:0.83894
[7]	validation_0-auc:0.83894
[8]	validation_0-auc:0.83893
[9]	validation_0-auc:0.84034
[10]	validation_0-auc:0.84039
[11]	validation_0-auc:0.84050
[12]	validation_0-auc:0.84253
[13]	validation_0-auc:0.84232
[14]	validation_0-auc:0.84498
[15]	validation_0-auc:0.84480
[16]	validation_0-auc:0.85226
[17]	validation_0-auc:0.85600
[18]	validation_0-auc:0.85483
[19]	validation_0-auc:0.85654
[20]	validation_0-auc:0.85790
[21]	validation_0-auc:0.85875
[22]	validation_0-auc:0.85816
[23]	validation_0-auc:0.85966
[24]	validation_0-auc:0.85827
[25]	validation_0-auc:0.85901
[26]	validation_0-auc:0.85829
[27]	validation_0-auc:0.85923
[28]	validation_0-auc:0.85864
[29]	validation_0-auc:0.86005
[30]	validation_0-auc:0.86053
[31]	validation_0-auc:0.86007
[32]	validation_0-auc:0.86175
[33]	validation_0-au

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[36]	validation_0-auc:0.86670
[37]	validation_0-auc:0.86638
[38]	validation_0-auc:0.86698
[39]	validation_0-auc:0.86606
[40]	validation_0-auc:0.86733
[41]	validation_0-auc:0.86840
[42]	validation_0-auc:0.86850
[43]	validation_0-auc:0.86948
[44]	validation_0-auc:0.87006
[45]	validation_0-auc:0.86917
[46]	validation_0-auc:0.87074
[47]	validation_0-auc:0.87101
[48]	validation_0-auc:0.87202
[49]	validation_0-auc:0.87282
[50]	validation_0-auc:0.87319
[51]	validation_0-auc:0.87330
[52]	validation_0-auc:0.87376
[53]	validation_0-auc:0.87356
[54]	validation_0-auc:0.87327
[55]	validation_0-auc:0.87427
[56]	validation_0-auc:0.87401
[57]	validation_0-auc:0.87412
[58]	validation_0-auc:0.87381
[59]	validation_0-auc:0.87380
[60]	validation_0-auc:0.87469
[61]	validation_0-auc:0.87460
[62]	validation_0-auc:0.87459
[63]	validation_0-auc:0.87524
[64]	validation_0-auc:0.87487
[65]	validation_0-auc:0.87466
[66]	validation_0-auc:0.87534
[67]	validation_0-auc:0.87516
[68]	validation_0-auc:0.87586
[69]	valid

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[29]	validation_0-auc:0.85620
[30]	validation_0-auc:0.85912
[31]	validation_0-auc:0.85932
[32]	validation_0-auc:0.86076
[33]	validation_0-auc:0.86281
[34]	validation_0-auc:0.86463
[35]	validation_0-auc:0.86449
[36]	validation_0-auc:0.86490
[37]	validation_0-auc:0.86732
[38]	validation_0-auc:0.86676
[39]	validation_0-auc:0.86842
[40]	validation_0-auc:0.86843
[41]	validation_0-auc:0.86879
[42]	validation_0-auc:0.86953
[43]	validation_0-auc:0.87001
[44]	validation_0-auc:0.86958
[45]	validation_0-auc:0.86812
[46]	validation_0-auc:0.86841
[47]	validation_0-auc:0.86835
[48]	validation_0-auc:0.86714
[49]	validation_0-auc:0.86732
[50]	validation_0-auc:0.86881
[51]	validation_0-auc:0.86915
[52]	validation_0-auc:0.87043
[53]	validation_0-auc:0.86945
[54]	validation_0-auc:0.86956
[55]	validation_0-auc:0.86986
[56]	validation_0-auc:0.87049
[57]	validation_0-auc:0.87044
[58]	validation_0-auc:0.87027
[59]	validation_0-auc:0.87080
[60]	validation_0-auc:0.87020
[61]	validation_0-auc:0.87051
[62]	valid

...

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[13]	validation_0-auc:0.84398
[14]	validation_0-auc:0.84125
[15]	validation_0-auc:0.85164
[16]	validation_0-auc:0.85211
[17]	validation_0-auc:0.85208
[18]	validation_0-auc:0.85439
[19]	validation_0-auc:0.85405
[20]	validation_0-auc:0.85548
[21]	validation_0-auc:0.85600
[22]	validation_0-auc:0.85972
[23]	validation_0-auc:0.85950
[24]	validation_0-auc:0.86416
[25]	validation_0-auc:0.86682
[26]	validation_0-auc:0.87040
[27]	validation_0-auc:0.87511
[28]	validation_0-auc:0.87202
[29]	validation_0-auc:0.87431
[30]	validation_0-auc:0.87650
[31]	validation_0-auc:0.87670
[32]	validation_0-auc:0.87696
[33]	validation_0-auc:0.87703
[34]	validation_0-auc:0.87595
[35]	validation_0-auc:0.87558
[36]	validation_0-auc:0.87490
[37]	validation_0-auc:0.87544
[38]	validation_0-auc:0.87594
[39]	validation_0-auc:0.87465
[40]	validation_0-auc:0.87539
[41]	validation_0-auc:0.87593
[42]	validation_0-auc:0.87582
[43]	validation_0-auc:0.87550
[44]	validation_0-auc:0.87571
[45]	validation_0-auc:0.87480
[46]	valid

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[21]	validation_0-auc:0.85494
[22]	validation_0-auc:0.85412
[23]	validation_0-auc:0.85465
[24]	validation_0-auc:0.85514
[25]	validation_0-auc:0.85841
[26]	validation_0-auc:0.85795
[27]	validation_0-auc:0.85733
[28]	validation_0-auc:0.85752
[29]	validation_0-auc:0.85918
[30]	validation_0-auc:0.86287
[31]	validation_0-auc:0.86524
[32]	validation_0-auc:0.86563
[33]	validation_0-auc:0.86640
[34]	validation_0-auc:0.86827
[35]	validation_0-auc:0.86960
[36]	validation_0-auc:0.87063
[37]	validation_0-auc:0.87207
[38]	validation_0-auc:0.87248
[39]	validation_0-auc:0.87255
[40]	validation_0-auc:0.87364
[41]	validation_0-auc:0.87381
[42]	validation_0-auc:0.87437
[43]	validation_0-auc:0.87557
[44]	validation_0-auc:0.87765
[45]	validation_0-auc:0.87877
[46]	validation_0-auc:0.87964
[47]	validation_0-auc:0.87984
[48]	validation_0-auc:0.87970
[49]	validation_0-auc:0.87975
[50]	validation_0-auc:0.88009
[51]	validation_0-auc:0.87974
[52]	validation_0-auc:0.87975
[53]	validation_0-auc:0.88010
[54]	valid

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                   

In [6]:
# 최적 파라미터 출력
print(model_grid.best_params_)
print(model_grid.best_score_)

{'eta': 0.1, 'max_depth': 4, 'n_estimators': 100, 'objective': 'binary:logistic'}
0.9568108386599145


In [7]:
# 최적 파라미터로 모델 적용 후 예측값 구하기
model = XGBClassifier(n_estimators=100, eta=0.1, max_depth=4, objective='binary:logistic')
model.fit(x_train, y_train, early_stopping_rounds=30, eval_metric='auc', eval_set = [(x_valid, y_valid)])

pro = pd.DataFrame(model.predict_proba(x_train))
y_reg= pd.DataFrame(pro.iloc[:, 1:2])

[0]	validation_0-auc:0.83221
[1]	validation_0-auc:0.83252
[2]	validation_0-auc:0.83241
[3]	validation_0-auc:0.83916
[4]	validation_0-auc:0.83954
[5]	validation_0-auc:0.83967
[6]	validation_0-auc:0.84306
[7]	validation_0-auc:0.84381
[8]	validation_0-auc:0.84380
[9]	validation_0-auc:0.84356
[10]	validation_0-auc:0.84391
[11]	validation_0-auc:0.84453
[12]	validation_0-auc:0.84573
[13]	validation_0-auc:0.84658
[14]	validation_0-auc:0.84816


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[15]	validation_0-auc:0.84843
[16]	validation_0-auc:0.85123
[17]	validation_0-auc:0.85368
[18]	validation_0-auc:0.85329
[19]	validation_0-auc:0.85225
[20]	validation_0-auc:0.85394
[21]	validation_0-auc:0.85494
[22]	validation_0-auc:0.85412
[23]	validation_0-auc:0.85465
[24]	validation_0-auc:0.85514
[25]	validation_0-auc:0.85841
[26]	validation_0-auc:0.85795
[27]	validation_0-auc:0.85733
[28]	validation_0-auc:0.85752
[29]	validation_0-auc:0.85918
[30]	validation_0-auc:0.86287
[31]	validation_0-auc:0.86524
[32]	validation_0-auc:0.86563
[33]	validation_0-auc:0.86640
[34]	validation_0-auc:0.86827
[35]	validation_0-auc:0.86960
[36]	validation_0-auc:0.87063
[37]	validation_0-auc:0.87207
[38]	validation_0-auc:0.87248
[39]	validation_0-auc:0.87255
[40]	validation_0-auc:0.87364
[41]	validation_0-auc:0.87381
[42]	validation_0-auc:0.87437
[43]	validation_0-auc:0.87557
[44]	validation_0-auc:0.87765
[45]	validation_0-auc:0.87877
[46]	validation_0-auc:0.87964
[47]	validation_0-auc:0.87984
[48]	valid

In [8]:
# Xgboost 회귀분석 모델 GridCV로 최적 파리미터 탐색
xgb_reg = XGBRegressor()
parameters = {'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'subsample': [0.4, 0.6,0.8],
              'colsample_bytree': [0.2, 0.6, 0.8],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb_reg,
                        parameters,
                        cv = 3
                        )
x_train2, x_test2, y_train2, y_test2 = train_test_split(x_train[test_stage_features], y_reg, test_size=0.2)
xgb_grid.fit(x_train2,y_train2)

GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=Non

In [9]:
# 최적 파라미터 출력
print(xgb_grid.best_params_)
print(xgb_grid.best_score_)

{'colsample_bytree': 0.6, 'learning_rate': 0.03, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 500, 'subsample': 0.8}
0.06629700996271463


In [10]:
# 최적 파라미터로 회귀분석에 학습 
model_reg = XGBRegressor(colsample_bytree= 0.6, learning_rate= 0.03, max_depth= 5, min_child_weight= 4, n_estimators= 500, nthread= 4, subsample= 0.8)
model_reg.fit(x_train2, y_train2, early_stopping_rounds=30, eval_metric='rmse', eval_set = [(x_test2, y_test2)])

[0]	validation_0-rmse:0.44851
[1]	validation_0-rmse:0.43765
[2]	validation_0-rmse:0.42723
[3]	validation_0-rmse:0.41715
[4]	validation_0-rmse:0.40749
[5]	validation_0-rmse:0.39814
[6]	validation_0-rmse:0.38919
[7]	validation_0-rmse:0.38071
[8]	validation_0-rmse:0.37243
[9]	validation_0-rmse:0.36445
[10]	validation_0-rmse:0.35677
[11]	validation_0-rmse:0.34931
[12]	validation_0-rmse:0.34224
[13]	validation_0-rmse:0.33540
[14]	validation_0-rmse:0.32889
[15]	validation_0-rmse:0.32270
[16]	validation_0-rmse:0.31667
[17]	validation_0-rmse:0.31089
[18]	validation_0-rmse:0.30541
[19]	validation_0-rmse:0.30020
[20]	validation_0-rmse:0.29514
[21]	validation_0-rmse:0.29026
[22]	validation_0-rmse:0.28565
[23]	validation_0-rmse:0.28120


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[24]	validation_0-rmse:0.27711
[25]	validation_0-rmse:0.27312
[26]	validation_0-rmse:0.26938
[27]	validation_0-rmse:0.26567
[28]	validation_0-rmse:0.26222
[29]	validation_0-rmse:0.25885
[30]	validation_0-rmse:0.25566
[31]	validation_0-rmse:0.25271
[32]	validation_0-rmse:0.24984
[33]	validation_0-rmse:0.24711
[34]	validation_0-rmse:0.24455
[35]	validation_0-rmse:0.24206
[36]	validation_0-rmse:0.23977
[37]	validation_0-rmse:0.23753
[38]	validation_0-rmse:0.23544
[39]	validation_0-rmse:0.23349
[40]	validation_0-rmse:0.23163
[41]	validation_0-rmse:0.22982
[42]	validation_0-rmse:0.22817
[43]	validation_0-rmse:0.22653
[44]	validation_0-rmse:0.22503
[45]	validation_0-rmse:0.22357
[46]	validation_0-rmse:0.22220
[47]	validation_0-rmse:0.22087
[48]	validation_0-rmse:0.21957
[49]	validation_0-rmse:0.21843
[50]	validation_0-rmse:0.21732
[51]	validation_0-rmse:0.21628
[52]	validation_0-rmse:0.21531
[53]	validation_0-rmse:0.21438
[54]	validation_0-rmse:0.21357
[55]	validation_0-rmse:0.21278
[56]	val

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.6,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.03, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=5, max_leaves=0, min_child_weight=4, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=4, nthread=4,
             num_parallel_tree=1, predictor='auto', ...)

In [11]:
# 모델에 test데이터 적용

y_hat = model_reg.predict(df_test)

y_hat = pd.DataFrame(y_hat)
y_hat

,0
0,0.051543
1,0.087613
2,0.124775
3,0.082955
4,0.174782
...,...
6036,0.034779
6037,0.064419
6038,0.104321
6039,0.073150


In [12]:
submit = pd.read_csv("src/data/test.csv")
submit['Y_LABEL'] = y_hat

final_submit = pd.DataFrame()
final_submit['ID'] = submit['ID']
final_submit['Y_LABEL'] = y_hat


def get_reg(v):
  if v >= 0.5:
    score = 1
  else:
    score = 0
  return score

final_submit['Y_LABEL'] = final_submit['Y_LABEL'].apply(lambda v: get_reg(v))
final_submit

final_submit.to_csv('src/data/submit4.csv', index=False)